##  FASE 1: ANÁLISIS INICIAL

### Paso 1.1 – Cargar el CSV

In [27]:
# Cargar CSV
import pandas as pd

def cargar_csv_formato_resenas(path):
    textos = []
    etiquetas = []
    with open(path, "r", encoding="utf-8") as f:
        next(f)  # saltar encabezado
        for linea in f:
            partes = linea.strip().rsplit(",", 1)  # dividir por última coma
            if len(partes) == 2:
                texto, etiqueta = partes
                textos.append(texto.strip())
                etiquetas.append(etiqueta.strip())
    return pd.DataFrame({"texto": textos, "etiqueta": etiquetas})

df = cargar_csv_formato_resenas("comentarios_clientes.csv")
df.head()


,texto,etiqueta
0,"El paquete llegó con el embalaje comprometido,...",neutral
1,"Después de 3 semanas de espera, el artículo nu...",negativo
2,La atención al cliente fue evasiva y poco reso...,negativo
3,Entrega express y producto en perfecto estado....,positivo
4,El sistema de seguimiento en línea mostró info...,negativo


In [28]:
# Ver resumen de valores faltantes por columna
print(df.isna().sum())

texto       0
etiqueta    0
dtype: int64


In [29]:
import spacy
import pandas as pd

# Cargar modelo spaCy en español
nlp = spacy.load("es_core_news_sm")

# Palabras cuya eliminación podría alterar el significado
# Detectar solo las críticas dentro del set completo de stopwords
stopwords_spacy = nlp.Defaults.stop_words
stopwords_criticas = {w for w in stopwords_spacy if w in {"no", "nunca", "tampoco", "pero", "aunque", "sin embargo"}}


# Función que evalúa si se eliminó una stopword crítica
def analizar_stopwords_criticas(fila):
    texto = fila["texto"]
    etiqueta = fila["etiqueta"]
    doc = nlp(texto)
    eliminadas = [t.text.lower() for t in doc if t.is_stop]
    eliminadas_criticas = [pal for pal in eliminadas if pal in stopwords_criticas]
    return {
        "texto": texto,
        "etiqueta": etiqueta,
        "eliminadas_criticas": eliminadas_criticas,
        "cambio_significado": len(eliminadas_criticas) > 0
    }

# Aplicar a todo el dataset
resultados_df = df.apply(analizar_stopwords_criticas, axis=1, result_type="expand")

# Mostrar tabla de verificación
tabla_verificacion = resultados_df[["texto", "etiqueta", "eliminadas_criticas", "cambio_significado"]]
print(tabla_verificacion.head(10))


                                               texto  etiqueta  \
0  El paquete llegó con el embalaje comprometido,...   neutral   
1  Después de 3 semanas de espera, el artículo nu...  negativo   
2  La atención al cliente fue evasiva y poco reso...  negativo   
3  Entrega express y producto en perfecto estado....  positivo   
4  El sistema de seguimiento en línea mostró info...  negativo   
5  Materiales de calidad inferior a lo descrito e...  negativo   
6  El proceso de devolución resultó engorroso por...  negativo   
7  Artículo funcional pero con manual de instrucc...   neutral   
8  Demoras recurrentes en la actualización del es...  negativo   
9  Embalaje ecológico y resistente que protegió b...  positivo   

  eliminadas_criticas  cambio_significado  
0            [aunque]                True  
1             [nunca]                True  
2                  []               False  
3                  []               False  
4                  []               False  
5        

## Fase 2: Personalización de la Lista

In [30]:
import spacy
import pandas as pd
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

# Asegurarse de tener los recursos de nltk descargados
nltk.download('stopwords')

# Cargar spaCy y nltk stopwords
nlp = spacy.load("es_core_news_sm")
stopwords_nltk = set(stopwords.words("spanish"))

# Paso 1: Tokenizar y contar frecuencia de palabras
todas_las_palabras = []
for texto in df["texto"]:
    tokens = [token.text.lower() for token in nlp(texto) if token.is_alpha]
    todas_las_palabras.extend(tokens)

frecuencia = Counter(todas_las_palabras)

# Paso 2: Construir stopwords personalizadas
stopwords_personalizadas = stopwords_nltk.copy()

# 2.1 – Eliminar términos clave que deben conservarse
terminos_a_conservar = {"no", "nunca", "tampoco", "pero", "aunque", "sin embargo"}
stopwords_personalizadas -= terminos_a_conservar

# 2.2 – Añadir términos genéricos redundantes y no informativos
t_terminos_genericos = {"producto", "cliente", "día", "hacer", "tener", "decir", "pd"}
stopwords_personalizadas |= t_terminos_genericos

# 2.3 – Añadir palabras como "hola" y "gracias" si aparecen en el texto usando regex
terminos_regex = re.compile(r"\b(hola|gracias)\b", flags=re.IGNORECASE)
for texto in df["texto"]:
    if terminos_regex.search(texto):
        for palabra in ["hola", "gracias"]:
            stopwords_personalizadas.add(palabra)

# Función para limpiar texto usando la lista personalizada
def limpiar_con_stopwords_personalizadas(texto):
    doc = nlp(texto)
    tokens_filtrados = [
        t.text.lower() for t in doc
        if t.text.lower() not in stopwords_personalizadas and t.is_alpha
    ]
    return tokens_filtrados

# Ejemplo de uso
print("Ejemplo 1")
ejemplo = df["texto"].iloc[0]
print("Original:", ejemplo)
print("Filtrado:", limpiar_con_stopwords_personalizadas(ejemplo))

print("\nEjemplo 2")
ejemplo = df["texto"].iloc[2]
print("Original:", ejemplo)
print("Filtrado:", limpiar_con_stopwords_personalizadas(ejemplo))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alumno_AI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Ejemplo 1
Original: El paquete llegó con el embalaje comprometido, aunque lograron reembolsarme rápidamente
Filtrado: ['paquete', 'llegó', 'embalaje', 'comprometido', 'aunque', 'lograron', 'reembolsarme', 'rápidamente']

Ejemplo 2
Original: La atención al cliente fue evasiva y poco resolutiva en mi reclamo
Filtrado: ['atención', 'evasiva', 'resolutiva', 'reclamo']


## Fase 3: Implementación y Pruebas

In [31]:
import spacy
import nltk
from nltk.corpus import stopwords

# Asegurar descarga de stopwords
nltk.download("stopwords")

# Cargar modelo y stopwords base
nlp = spacy.load("es_core_news_sm")
stopwords_es = set(stopwords.words("spanish"))

# Quitar términos críticos que deben preservarse
terminos_clave = {"no", "nunca", "tampoco", "pero", "aunque", "sin embargo"}
stopwords_es -= terminos_clave

# Añadir términos no informativos y genéricos
stopwords_es |= {"producto", "cliente", "día", "hacer", "tener", "decir", "hola", "gracias", "pd", "tan"}

# Función de procesamiento
def procesar_texto(texto):
    doc = nlp(texto)
    tokens = [
        token.text.lower()
        for token in doc
        if token.is_alpha and token.text.lower() not in stopwords_es
    ]
    return tokens

# Casos de prueba
casos = {
    "Texto 1": "No funciona bien, pero el diseño es bonito.",
    "Texto 2": "Nunca compré algo tan malo. Aunque el precio es bajo, no lo vale."
}

# Resultados esperados
esperados = {
    "Texto 1": ["no", "funciona", "bien", "pero", "diseño", "bonito"],
    "Texto 2": ["nunca", "compré", "malo", "aunque", "precio", "bajo", "no", "vale"]
}

# Prueba
print("Resultados de Casos de Prueba:\n")
for nombre, texto in casos.items():
    tokens = procesar_texto(texto)
    print(f"{nombre}:\n  Original: {texto}\n  Tokens procesados: {tokens}")
    print(f"  Esperado: {esperados[nombre]}")
    print(f"  ✅ Correcto: {tokens == esperados[nombre]}\n")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alumno_AI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Resultados de Casos de Prueba:

Texto 1:
  Original: No funciona bien, pero el diseño es bonito.
  Tokens procesados: ['no', 'funciona', 'bien', 'pero', 'diseño', 'bonito']
  Esperado: ['no', 'funciona', 'bien', 'pero', 'diseño', 'bonito']
  ✅ Correcto: True

Texto 2:
  Original: Nunca compré algo tan malo. Aunque el precio es bajo, no lo vale.
  Tokens procesados: ['nunca', 'compré', 'malo', 'aunque', 'precio', 'bajo', 'no', 'vale']
  Esperado: ['nunca', 'compré', 'malo', 'aunque', 'precio', 'bajo', 'no', 'vale']
  ✅ Correcto: True



## Fase 4: Evaluación de Impacto

In [33]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar modelo spaCy
nlp = spacy.load("es_core_news_sm")

# Crear lista personalizada de stopwords
stopwords_es = nlp.Defaults.stop_words.copy()
terminos_a_conservar = {"no", "nunca", "tampoco", "pero", "aunque", "sin embargo"}
stopwords_adicionales = {"producto", "cliente", "día", "hacer", "tener", "decir", "hola", "gracias", "pd"}

for palabra in terminos_a_conservar:
    stopwords_es.discard(palabra)
stopwords_es.update(stopwords_adicionales)

# Función para limpiar texto con stopwords personalizadas
def limpiar_con_stopwords(texto):
    doc = nlp(texto)
    tokens = [token.text.lower() for token in doc if token.is_alpha and token.text.lower() not in stopwords_es]
    return " ".join(tokens)

# Aplicar limpieza al dataset
df_filtrado = df.copy()
df_filtrado["texto_filtrado"] = df_filtrado["texto"].apply(limpiar_con_stopwords)

# Submuestreo si es necesario
df_sub = df_filtrado.sample(n=200, random_state=42) if len(df_filtrado) > 200 else df_filtrado

# ----------- MODELO BASE (sin stopwords personalizadas) -----------
X_train, X_test, y_train, y_test = train_test_split(df_sub["texto"], df_sub["etiqueta"], test_size=0.3, random_state=42)

vectorizer_std = CountVectorizer(max_features=1000)
X_train_vec = vectorizer_std.fit_transform(X_train)
X_test_vec = vectorizer_std.transform(X_test)

modelo_std = LogisticRegression(max_iter=1000)
modelo_std.fit(X_train_vec, y_train)
preds_std = modelo_std.predict(X_test_vec)
acc_std = accuracy_score(y_test, preds_std)

# ----------- MODELO PERSONALIZADO (con stopwords personalizadas) -----------
X_train_f, X_test_f, _, _ = train_test_split(df_sub["texto_filtrado"], df_sub["etiqueta"], test_size=0.3, random_state=42)

vectorizer_custom = CountVectorizer(max_features=1000)
X_train_vec_f = vectorizer_custom.fit_transform(X_train_f)
X_test_vec_f = vectorizer_custom.transform(X_test_f)

modelo_custom = LogisticRegression(max_iter=1000)
modelo_custom.fit(X_train_vec_f, y_train)
preds_custom = modelo_custom.predict(X_test_vec_f)
acc_custom = accuracy_score(y_test, preds_custom)

# ----------- RESULTADOS -----------

print(f"\nAccuracy SIN stopwords personalizadas: {acc_std:.2f}")
print(f"Accuracy CON stopwords personalizadas: {acc_custom:.2f}")

# Comparación de predicciones
comparacion = pd.DataFrame({
    "original": X_test.values,
    "filtrado": X_test_f.values,
    "etiqueta": y_test.values,
    "pred_sin_personalizadas": preds_std,
    "pred_con_personalizadas": preds_custom
})

diferencias = comparacion[comparacion["pred_sin_personalizadas"] != comparacion["pred_con_personalizadas"]]
print("\nMuestras con diferencias de predicción:")
print(diferencias.head())

# Análisis de impacto
mejoraron = ((preds_std != y_test) & (preds_custom == y_test)).sum()
empeoraron = ((preds_std == y_test) & (preds_custom != y_test)).sum()
sin_cambios = (preds_std == preds_custom).sum()

print(f"\n🔍 Análisis de impacto:")
print(f"  ✅ Mejoraron con stopwords personalizadas: {mejoraron}")
print(f"  ❌ Empeoraron con stopwords personalizadas: {empeoraron}")
print(f"  ➖ Sin cambios en la predicción: {sin_cambios}")



Accuracy SIN stopwords personalizadas: 0.53
Accuracy CON stopwords personalizadas: 0.40

Muestras con diferencias de predicción:
                                             original  \
7   Experiencia unboxing premium con empaque de di...   
11  El sistema de seguimiento en línea mostró info...   

                                             filtrado  etiqueta  \
7   experiencia unboxing premium empaque diseño cu...  positivo   
11  sistema seguimiento línea mostró información i...  negativo   

   pred_sin_personalizadas pred_con_personalizadas  
7                 positivo                negativo  
11                negativo                positivo  

🔍 Análisis de impacto:
  ✅ Mejoraron con stopwords personalizadas: 0
  ❌ Empeoraron con stopwords personalizadas: 2
  ➖ Sin cambios en la predicción: 13


### Guardar lista

In [34]:
# Guardar lista de stopwords personalizadas en un archivo .txt
with open("stopwords_personalizadas.txt", "w", encoding="utf-8") as f:
    for palabra in sorted(stopwords_es):
        f.write(palabra + "\n")
